<a href="https://colab.research.google.com/github/NzolaKiampava/python-demoapp/blob/master/kimakudi_credits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# instalar
!pip install pandas numpy

#importar
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [ ]:
def generate_credit_dataset(num_records=1000):
    np.random.seed(42)  # Para reprodutibilidade

    # Dados básicos (mantido do código anterior)
    data = {
        'id_cliente': range(1, num_records + 1),
        'idade': np.random.randint(18, 75, num_records),
        'renda_mensal': np.random.normal(5000, 2000, num_records),
        'tempo_emprego_anos': np.random.randint(0, 30, num_records),
        'score_credito': np.random.randint(300, 850, num_records),
        'possui_imovel': np.random.choice([0, 1], num_records, p=[0.7, 0.3]),
        'possui_veiculo': np.random.choice([0, 1], num_records, p=[0.6, 0.4]),
        'estado_civil': np.random.choice(['Solteiro', 'Casado', 'Divorciado', 'Viúvo'], num_records),
        'dependentes': np.random.randint(0, 5, num_records),
        'valor_solicitado': np.random.uniform(1000, 50000, num_records),
        'prazo_meses': np.random.choice([12, 24, 36, 48, 60], num_records),
        'finalidade': np.random.choice([
            'Reforma', 'Educação', 'Compra de Veículo',
            'Pagamento de Dívidas', 'Investimento', 'Outros'
        ], num_records),
        'dividas_atuais': np.random.uniform(0, 20000, num_records),
        'qtd_emprestimos_ativos': np.random.randint(0, 4, num_records)
    }

    # Criar DataFrame
    df = pd.DataFrame(data)

    # Ajustes básicos (mantido do código anterior)
    df['renda_mensal'] = df['renda_mensal'].round(2).clip(lower=1100)
    df['valor_solicitado'] = df['valor_solicitado'].round(2)
    df['dividas_atuais'] = df['dividas_atuais'].round(2)
    df['comprometimento_renda'] = ((df['dividas_atuais'] / 12) / df['renda_mensal'] * 100).round(2)

    # Histórico de Crédito Expandido

    # 1. Histórico de pagamentos dos últimos 24 meses
    for i in range(1, 25):
        df[f'pagamento_mes_{i}'] = np.random.choice(
            ['Em dia', 'Atraso 1-15 dias', 'Atraso 16-30 dias', 'Atraso >30 dias'],
            num_records,
            p=[0.85, 0.08, 0.05, 0.02]
        )

    # 2. Histórico de empréstimos anteriores
    df['qtd_emprestimos_anteriores'] = np.random.randint(0, 6, num_records)
    df['valor_total_emprestimos_anteriores'] = np.random.uniform(0, 100000, num_records).round(2)
    df['qtd_emprestimos_pagos_pontualmente'] = df.apply(
        lambda x: np.random.randint(0, max(1, x['qtd_emprestimos_anteriores'] + 1)),
        axis=1
    )

    # 3. Histórico de consultas de crédito
    df['consultas_credito_30dias'] = np.random.randint(0, 5, num_records)
    df['consultas_credito_6meses'] = df['consultas_credito_30dias'] + np.random.randint(0, 8, num_records)
    df['consultas_credito_12meses'] = df['consultas_credito_6meses'] + np.random.randint(0, 10, num_records)

    # 4. Relacionamento com bancos
    df['tempo_relacionamento_bancario'] = np.random.randint(0, 240, num_records)  # em meses
    df['qtd_contas_bancarias'] = np.random.randint(1, 5, num_records)
    df['possui_investimentos'] = np.random.choice([0, 1], num_records, p=[0.7, 0.3])
    df['valor_medio_investimentos'] = df['possui_investimentos'] * np.random.uniform(0, 50000, num_records).round(2)

    # 5. Histórico de cartão de crédito
    df['possui_cartao_credito'] = np.random.choice([0, 1], num_records, p=[0.2, 0.8])
    df['limite_cartao_total'] = df['possui_cartao_credito'] * np.random.uniform(500, 25000, num_records).round(2)
    df['utilizacao_media_cartao'] = np.random.uniform(0, 100, num_records).round(2)  # em porcentagem
    df['atrasos_cartao_12meses'] = np.random.randint(0, 5, num_records)

    # 6. Histórico de negativação
    df['teve_restricao'] = np.random.choice([0, 1], num_records, p=[0.85, 0.15])
    df['dias_ultima_restricao'] = df['teve_restricao'] * np.random.randint(0, 1825, num_records)  # até 5 anos
    df['valor_ultima_restricao'] = df['teve_restricao'] * np.random.uniform(0, 10000, num_records).round(2)

    # Calcular taxa de juros com fatores adicionais
    df['taxa_juros'] = (
        (850 - df['score_credito']) / 850 * 5 +
        df['prazo_meses'] / 12 * 0.5 +
        df['teve_restricao'] * 2 +
        (df['atrasos_cartao_12meses'] / 4) +
        (df['utilizacao_media_cartao'] / 100) +
        np.random.uniform(-0.5, 0.5, num_records)
    ).round(2).clip(lower=0.5)

    return df

In [5]:
# Gerar dataset
df_credito = generate_credit_dataset()

# Mostrar as primeiras linhas e informações do dataset
print("Dimensões do dataset:", df_credito.shape)
print("\nPrimeiras linhas do dataset:")
print(df_credito.head())
print("\nInformações do dataset:")
print(df_credito.info())

# salvar o dataset em um arquivo csv
df_credito.to_csv('df_credito.csv', index=False)

# para fazer download do arquivo
from google.colab import files
files.download('df_credito.csv')

Dimensões do dataset: (1000, 57)

Primeiras linhas do dataset:
   id_cliente  idade  renda_mensal  tempo_emprego_anos  score_credito  \
0           1     56       1100.00                  10            589   
1           2     69       4821.76                  21            739   
2           3     46       2391.06                  21            306   
3           4     32       6339.35                  10            495   
4           5     60       5733.20                  24            720   

   possui_imovel  possui_veiculo estado_civil  dependentes  valor_solicitado  \
0              0               1     Solteiro            1          17937.19   
1              1               1   Divorciado            1          37421.43   
2              0               0        Viúvo            3          22467.47   
3              1               1     Solteiro            1          41071.63   
4              1               1     Solteiro            2          27916.99   

   ...  possui_in

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>